# Week 3 of IBM Advanced Data Science Capstone Project

## For this section, I will apply simple predictive modeling using PySpark ML before using the algorithms in Python sci-kit learn library.

### Import csv file from previous week


In [1]:
!wget https://raw.githubusercontent.com/aziz0519/Advanced-Data-Science-Capstone-Project/master/listing_model.csv

--2020-03-22 12:23:06--  https://raw.githubusercontent.com/aziz0519/Advanced-Data-Science-Capstone-Project/master/listing_model.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417650 (408K) [text/plain]
Saving to: ‘listing_model.csv’

listing_model.csv   100%[===================>] 407.86K  --.-KB/s    in 0.06s   

2020-03-22 12:23:11 (6.78 MB/s) - ‘listing_model.csv’ saved [417650/417650]



In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
data = pd.read_csv('listing_model.csv')
data.head()

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,distance_to_nearest_stn
0,2,42,1.310995,103.79580,1,84,180,1,2,365,0.617939
1,0,6,1.332350,103.78521,1,80,90,18,1,365,0.397868
2,2,42,1.310995,103.79667,1,70,6,20,2,365,0.524388
3,1,37,1.345410,103.95712,1,207,1,20,8,365,0.486465
4,1,37,1.345670,103.95963,1,95,1,24,8,365,0.479073


In [5]:
data.columns

Index(['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365',
       'distance_to_nearest_stn'],
      dtype='object')

In [0]:
#Drop columns that does not contain any number of reviews
#data_final = data[data.number_of_reviews > 0]
#data_final.shape

In [0]:
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [0]:
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.linear_model import Ridge, RidgeCV , ElasticNet , ElasticNetCV
from sklearn.svm import SVR   

In [0]:
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold ,cross_val_score
from sklearn.metrics import f1_score , roc_auc_score

In [0]:
X = data[['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'minimum_nights', 'number_of_reviews',
       'calculated_host_listings_count', 'availability_365',
       'distance_to_nearest_stn']]
y = data['price']

In [0]:
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [0]:
X_train, X_test , y_train , y_test = train_test_split(X,y, test_size=0.33, random_state=42)

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred = lr.predict(X_test)

In [18]:
y_test

8006    168
737     130
3014    101
7512    120
3105     60
       ... 
4845    150
6803    160
3148    600
6187    101
3948    129
Name: price, Length: 2654, dtype: int64

In [24]:
prediction = lr.predict(X_test)
error_airbnb = pd.DataFrame({
        'Actual Values': np.array(y_test).flatten(),
        'Predicted Values': prediction.flatten()})
error_airbnb.head()

,Actual Values,Predicted Values
0,168,171.345585
1,130,190.093650
2,101,167.762828
3,120,254.822666
4,60,213.719478
